In [1]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import finnhub
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# change to your api keys 

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'CORYDT1RMNVL3DVT'
polygon_api_key = 'J9UUfkyTylf3D_s76b5pcMCnLSaKTNpD'
finnhub_client = finnhub.Client(api_key="cld35g9r01qn959jif1gcld35g9r01qn959jif20")


### Screening US Mega & Large Cap Stocks

In [3]:
file_path = 'nasdaq_screener_1700335987448.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'nasdaq_screener_1700335987448.csv'

In [ ]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

### Generating functions to retrieve stock information 

In [ ]:
# define function to generate random date, and ensure that the date is a weekday, maybe start in 2014 or 2015

def generate_random_date(start_date, end_date):
    check_weekday = False
    while not check_weekday:
        # Convert the date strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date_dt - start_date_dt).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date_dt + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            check_weekday = True
            return random_date
        else:
            check_weekday = False



In [ ]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock.lower()
    

In [ ]:
# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, check_date):
    check_financials = False
    # Check if the input date is a Friday
    if check_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = check_date + timedelta(days=3)
    else:
        # Set end date to the next day
        adj_end_date = check_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    
    
    try:
        get_data(ticker, end_date=adj_end_date)
        get_stock_financials(ticker, check_date)
        

        check_financials = True
    except:
        check_financials = False
    
    if not check_financials:
        return False
    else:
        return True
    

In [ ]:
# define a function to return stock industry

def return_stock_industry(symbol):
    symbol = symbol.upper()
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [ ]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, stock_date):
    # Check if the input date is a Friday
    if stock_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        price_adj_end_date = stock_date + timedelta(days=3)
    else:
        # Set end date to the next day
        price_adj_end_date = stock_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=price_adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [ ]:
# define a function to get stock trading volume

def get_stock_volume(ticker, vol_date):
    # Check if the input date is a Friday
    if vol_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        vol_adj_end_date = vol_date + timedelta(days=3)
    else:
        # Set end date to the next day
        vol_adj_end_date = vol_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=vol_adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [ ]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(weekday_date):
    previous_weekdays = []
    previous_date = weekday_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [ ]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, fin_date):
    
    financial_ratios = {}

    # setting brwoser
    browser = Browser('chrome')
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        # Extract date columns excluding the first column
        date_columns = df.columns[2:-1]
        
        # Convert date columns to datetime objects
        date_columns = pd.to_datetime(date_columns)
        
        # Find the closest date column to the given date that is earlier
        earlier_dates = [date for date in date_columns if date < fin_date]
        closest_date_column = max(earlier_dates)

        # Access the closest column
        closest_column = closest_date_column.strftime('%Y-%m-%d')
        
    except:
        print("Table not found on the page.")
    
    

    #extracting financial ratios
    try: 
        financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
        financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
        financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
        total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
        financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    except:
        pass 
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
    # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    except:
        print("Table not found on the page.")
    
    try:
        profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
        financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
        cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
        financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    except:
        pass

    return financial_ratios
    
    

In [ ]:
# function to calculate stock's volatility

def calculate_volatility(ticker, sd_date):
    df=get_data(ticker, end_date=sd_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [ ]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, news_date):
    
    # Extract year, month, and day
    news_end_date=news_date
    end_year = news_end_date.year
    end_month = str(news_end_date.month).zfill(2)
    end_day = str(news_end_date.day).zfill(2)
    
    news_start_date = news_end_date - timedelta(weeks=4)
    start_year = news_start_date.year
    start_month = str(news_start_date.month).zfill(2)
    start_day = str(news_start_date.day).zfill(2)
   

In [ ]:
# extracting daily federal interest rate information from alphavantage

# using alphavantage API
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

daily_interest_rates = data

# converting json to dataframe
daily_interest_rate_df = pd.DataFrame(daily_interest_rates['data'])

# Export DataFrame to CSV so that we can refer to it without making an api call
daily_interest_rate_df.to_csv('daily_interest_rates.csv', index=False)


In [ ]:
# extracting monthly CPI information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_cpi = data

# converting json to dataframe
monthly_cpi_df = pd.DataFrame(monthly_cpi['data'])
monthly_cpi_df['date'] = pd.to_datetime(monthly_cpi_df['date'])


# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_cpi_df.to_csv('monthly_cpi.csv', index=False)

In [ ]:
# extracting monthly unemployment rate information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_unemployment_rate = data

# converting json to dataframe
monthly_unemployment_rate_df = pd.DataFrame(monthly_unemployment_rate['data'])
monthly_unemployment_rate_df['date'] = pd.to_datetime(monthly_unemployment_rate_df['date'])

# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_unemployment_rate_df.to_csv('monthly_unemployment_rate.csv', index=False)

In [ ]:
# define a function to extract economic indicators for the given date

def get_economic_indicators(econ_date):
    
    economic_indicators_dict={}
    
    # convert datetime object to matching string 
    formatted_date = econ_date.strftime('%Y-%m-%d')
    
    # retrieve CPI value
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['cpi'] = float(monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])    
    
    # retrieve federal interest rate value
    economic_indicators_dict['interest_rate'] = float(daily_interest_rate_df.loc[daily_interest_rate_df['date'] == formatted_date, 'value'].iloc[0])

    # retrieve unemployement rate
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['unemployment_rate'] = float(monthly_unemployment_rate_df.loc[monthly_unemployment_rate_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])
    
    return economic_indicators_dict

In [ ]:
# function to retrieve technical indicators

def get_technical_indicators(ticker, tech_date):
    
    technical_indicators_dict={}
    
    # changing date into UNIX format
    end_timestamp = int(tech_date.timestamp())
    
    # choosing a timestamp of one week earlier
    start_tech_date = tech_date - timedelta(weeks=1)
    
    # changing the date to UNIX format
    start_timestamp = int(start_tech_date.timestamp())
    
    # retrieving SMA 
    technical_indicators_dict['sma']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='sma', indicator_fields={"timeperiod": 5})['sma'][-1]
    
    #retrieving RSI
    technical_indicators_dict['rsi']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='rsi', indicator_fields={"timeperiod": 5})['rsi'][-1]

    # retrieving EMA 
    technical_indicators_dict['ema']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='ema', indicator_fields={"timeperiod": 5})['ema'][-1]
    
    return technical_indicators_dict

In [ ]:
# function to create label value for data

def create_label(ticker, y_date):
    print(y_date)
    initial_price = get_stock_price(ticker, y_date)
    print(initial_price)
    
    # find the next weekday
    
    # check if date landed on friday and calculate the next weekday date
    if y_date.weekday() == 4:
        next_weekday_date = y_date + timedelta(days=3)
    else:
        next_weekday_date = y_date + timedelta(days=1)

    print(next_weekday_date)
    
    next_price = get_stock_price(ticker, next_weekday_date)
    
    print(next_price)
    
    if next_price >= initial_price:
        return 1 # 1 is class increase, in order to simplify the classification we put stayed the same in same category as increase
    else:
        return 0 # 0 is class decrease

### Running a loop to generate multiple rows of data for training and testing

In [ ]:
page=10

In [ ]:
# list to store dictionaries
list_dict = []
page += 1
# runs loop until list_dic [change no. 4 as desired]
while len(list_dict) < 500:

    # data_dict
    data_dict = {}
    
    # checking to see if generated ticker-date combo has available data
    checker = False
    
    # runs this loop until checker is true
    while not checker:

        ticker = choose_random_stock(ticker_list)
        input_date = generate_random_date('2015-01-01', '2023-10-31')

        checker = check_stock_date(ticker, input_date)

    # using function to return stock industry
    data_dict['industry']=return_stock_industry(ticker)

    # retrieve weekly returns
    date = input_date
    for i in range(1, 5):
        data_dict[f'wr{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
        date = date - timedelta(weeks=1)

    # retrieve last 5 working day volumes
    data_dict['vol1'] = get_stock_volume(ticker, date)
    for i, weekday in enumerate(get_previous_weekdays(input_date)):
        data_dict[f'vol{i+1}'] = get_stock_volume(ticker, weekday)

    # retrieve stock financials    
    for key, value in get_stock_financials(ticker, input_date).items():
        data_dict[key] = value
    
    # retrieve stock volatility
    data_dict['volatility'] = calculate_volatility(ticker, input_date)

    # retrieve economic indicators
    for key, value in get_economic_indicators(input_date).items():
        data_dict[key] = value

    # retrieve stocktechnical indicators    
    for key, value in get_technical_indicators(ticker, input_date).items():
        data_dict[key] = value

    # generate a label for row    
    data_dict['label'] = create_label(ticker, input_date)
        
    # append dictionary to list
    list_dict.append(data_dict)
    print(len(list_dict))

In [ ]:
data_dict

In [ ]:
print(ticker, input_date)

In [ ]:
pd.to_datetime(input_date).to_period('M')

In [ ]:
monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(input_date).to_period('M'), 'value'].iloc[0]  

In [ ]:
list_dict

### Export data into csv

In [ ]:
# remember to export data as csv
data_df = pd.DataFrame(list_dict)
data_df.to_csv(f'data{page}.csv', index=False)


In [ ]:
data_df

In [ ]:
page = 1

In [ ]:
print(ticker, input_date)